### Step 1. Create engine object, like normal.

In [5]:
from sqlalchemy import create_engine
import os

In [6]:
pws = os.getenv('DINAS_POSTGRES_PASSWORD')

In [9]:
conn_string = 'postgres://localhost:5432/northwind'

In [10]:
engine = create_engine(conn_string, encoding='latin1', echo=True)

In [13]:
next(engine.execute('SELECT * FROM customers'))

2020-03-12 09:46:09,316 INFO sqlalchemy.engine.base.Engine SELECT * FROM customers
2020-03-12 09:46:09,318 INFO sqlalchemy.engine.base.Engine {}


('ALFKI', 'Alfreds Futterkiste', 'Maria Anders', 'Sales Representative', 'Obere Str. 57', 'Berlin', None, '12209', 'Germany', '030-0074321', '030-0076545')

### Step 2. Declarative Mapping

In [14]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

### Step 3. Create a table Class that will map to an actual table in PostGres

In [15]:
from sqlalchemy import Column, Integer, String

class Movie(Base):
    
    __tablename__ = 'movie'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(50)) #VARCHAR(50)
    director = Column(String(50))
    genres = Column(String(100))
    
    def __repr__(self):
        return f'<Movie(id={self.id}, title={self.title})>'
    
    

### Step 4. Create the Table

In [16]:
Movie.__table__

Table('movie', MetaData(bind=None), Column('id', Integer(), table=<movie>, primary_key=True, nullable=False), Column('title', String(length=50), table=<movie>), Column('director', String(length=50), table=<movie>), Column('genres', String(length=100), table=<movie>), schema=None)

In [17]:
Base.metadata.create_all(engine)

2020-03-12 09:52:06,230 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-03-12 09:52:06,232 INFO sqlalchemy.engine.base.Engine {'name': 'movie'}
2020-03-12 09:52:06,267 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE movie (
	id SERIAL NOT NULL, 
	title VARCHAR(50), 
	director VARCHAR(50), 
	genres VARCHAR(100), 
	PRIMARY KEY (id)
)


2020-03-12 09:52:06,268 INFO sqlalchemy.engine.base.Engine {}
2020-03-12 09:52:06,336 INFO sqlalchemy.engine.base.Engine COMMIT


In [18]:
movie1 = Movie(title='Gladiator', director='Ridley Scott', genres = 'Drama|Action|Historical')

In [19]:
movie1

<Movie(id=None, title=Gladiator)>

### Step 5. Establish a "Session"

In [20]:
from sqlalchemy.orm import sessionmaker
#factory function from sqlalchemy

In [21]:
Session = sessionmaker(bind=engine)
session = Session() #kind of like git init

### Step 6. Add data to the session

In [22]:
session.add(movie1) #kind of like git add

In [23]:
movie2 = Movie(title='Toy Story', director='John Lasseter', genres='Children|Animation')
movie3 = Movie(title='Up', director='JPete Docter', genres='Children|Animation')
movie4 = Movie(title='Interstellar', director='Christopher Nolan', genres='Sci-Fi|Action')

In [24]:
session.add_all([movie2, movie3, movie4])

In [25]:
session.new #git status after adding 

IdentitySet([<Movie(id=None, title=Gladiator)>, <Movie(id=None, title=Toy Story)>, <Movie(id=None, title=Up)>, <Movie(id=None, title=Interstellar)>])

### Step 7. Commit data to the database

In [26]:
session.commit()

2020-03-12 09:58:54,713 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-12 09:58:54,719 INFO sqlalchemy.engine.base.Engine INSERT INTO movie (title, director, genres) VALUES (%(title)s, %(director)s, %(genres)s) RETURNING movie.id
2020-03-12 09:58:54,720 INFO sqlalchemy.engine.base.Engine {'title': 'Gladiator', 'director': 'Ridley Scott', 'genres': 'Drama|Action|Historical'}
2020-03-12 09:58:54,749 INFO sqlalchemy.engine.base.Engine INSERT INTO movie (title, director, genres) VALUES (%(title)s, %(director)s, %(genres)s) RETURNING movie.id
2020-03-12 09:58:54,750 INFO sqlalchemy.engine.base.Engine {'title': 'Toy Story', 'director': 'John Lasseter', 'genres': 'Children|Animation'}
2020-03-12 09:58:54,752 INFO sqlalchemy.engine.base.Engine INSERT INTO movie (title, director, genres) VALUES (%(title)s, %(director)s, %(genres)s) RETURNING movie.id
2020-03-12 09:58:54,754 INFO sqlalchemy.engine.base.Engine {'title': 'Up', 'director': 'JPete Docter', 'genres': 'Children|Animation'

How to query:

In [27]:
result = session.query(Movie).filter(Movie.id > 1).filter(Movie.title.like('%T%'))

In [29]:
for i in result:
    print(i.title)

2020-03-12 10:07:28,317 INFO sqlalchemy.engine.base.Engine SELECT movie.id AS movie_id, movie.title AS movie_title, movie.director AS movie_director, movie.genres AS movie_genres 
FROM movie 
WHERE movie.id > %(id_1)s AND movie.title LIKE %(title_1)s
2020-03-12 10:07:28,318 INFO sqlalchemy.engine.base.Engine {'id_1': 1, 'title_1': '%T%'}
Toy Story


In [30]:
result = session.query(Movie).filter_by(title='Gladiator').first()

2020-03-12 10:07:50,383 INFO sqlalchemy.engine.base.Engine SELECT movie.id AS movie_id, movie.title AS movie_title, movie.director AS movie_director, movie.genres AS movie_genres 
FROM movie 
WHERE movie.title = %(title_1)s 
 LIMIT %(param_1)s
2020-03-12 10:07:50,385 INFO sqlalchemy.engine.base.Engine {'title_1': 'Gladiator', 'param_1': 1}


In [31]:
result.title

'Gladiator'

### Step 8. What about tables that already exist?

In [ ]:
engine2 = create_engine('postgres://localhost:5432/movies-small', echo=True)


In [ ]:
Base2 = declarative_base(bind=engine2)

In [ ]:
class Rating(Base2):
    
    __tablename__ = 'ratings'
    __table_args__ = {'autoload': True}

In [ ]:
Session2 = sessionmaker(bind=engine2)
session2 = Session2()

In [ ]:
example = session2.query(Rating).first() #SELECT * FROM ratings LIMIT 1;

In [ ]:
example.movieId